In [ ]:
# Import library
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive
from IPython.display import display, Image

# Mount Google Drive
drive.mount('/content/drive')

# Konfigurasi
BASE_DIR = '/content/drive/MyDrive/autoencoder_project'  # Ganti dengan path Anda
INPUT_DIR = os.path.join(BASE_DIR, 'data/input')
OUTPUT_DIR = os.path.join(BASE_DIR, 'data/output')
MODEL_DIR = os.path.join(BASE_DIR, 'models')
RESULT_DIR = os.path.join(BASE_DIR, 'results')
IMG_SIZE = (256, 256)
BATCH_SIZE = 8
EPOCHS = 30

# Buat folder jika belum ada
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_data():
    """Muat pasangan gambar input-output"""
    input_images = []
    output_images = []

    input_files = sorted(os.listdir(INPUT_DIR))
    output_files = sorted(os.listdir(OUTPUT_DIR))

    print(f"Ditemukan {len(input_files)} gambar input dan {len(output_files)} gambar output")

    for in_file, out_file in zip(input_files, output_files):
        # Baca gambar
        in_img = cv2.imread(os.path.join(INPUT_DIR, in_file))
        out_img = cv2.imread(os.path.join(OUTPUT_DIR, out_file))

        if in_img is None or out_img is None:
            print(f"Melewati {in_file}/{out_file} - Gambar rusak")
            continue

        # Konversi dan resize
        in_img = cv2.cvtColor(in_img, cv2.COLOR_BGR2RGB)
        out_img = cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB)

        in_img = cv2.resize(in_img, IMG_SIZE) / 255.0
        out_img = cv2.resize(out_img, IMG_SIZE) / 255.0

        input_images.append(in_img)
        output_images.append(out_img)

    return train_test_split(
        np.array(input_images),
        np.array(output_images),
        test_size=0.2,
        random_state=42
    )

In [ ]:
def build_autoencoder(input_shape):
    """Bangun model autoencoder konvolusional"""
    # Encoder
    inputs = Input(input_shape)

    # Downsampling
    x = Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2,2), padding='same')(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), padding='same')(x)

    x = Conv2D(256, (3,3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2,2), padding='same')(x)

    # Decoder
    x = Conv2D(256, (3,3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2,2))(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)

    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)

    decoded = Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

    # Kompilasi model
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer=Adam(0.001), loss='mse')

    return autoencoder

In [ ]:
def train_model():
    """Proses training model"""
    # Muat data
    X_train, X_val, y_train, y_val = load_data()

    # Bangun model
    model = build_autoencoder(X_train[0].shape)
    model.summary()

    # Callbacks
    callbacks = [
        ModelCheckpoint(
            os.path.join(MODEL_DIR, 'autoencoder_best.h5'),
            save_best_only=True,
            monitor='val_loss'
        ),
        EarlyStopping(
            patience=5,
            restore_best_weights=True
        )
    ]

    # Training
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=callbacks
    )

    # Plot history
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.savefig(os.path.join(RESULT_DIR, 'training_history.png'))
    plt.show()

    # Simpan model final
    model.save(os.path.join(MODEL_DIR, 'autoencoder_final.h5'))
    print("Training selesai! Model disimpan di:", MODEL_DIR)

In [ ]:
def test_image(image_path):
    """Uji model dengan gambar baru"""
    # Muat model
    try:
        model = tf.keras.models.load_model(
            os.path.join(MODEL_DIR, 'autoencoder_best.h5'),
            compile=False
        )
    except:
        raise ValueError("Model tidak ditemukan. Jalankan training terlebih dahulu!")

    # Preprocess gambar
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Gagal membaca gambar di {image_path}")

    original = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(original, IMG_SIZE)
    img = (img.astype('float32') / 255.0).reshape(1, *IMG_SIZE, 3)

    # Prediksi
    output = model.predict(img)[0]
    output = (output * 255).astype('uint8')
    output = cv2.resize(output, (original.shape[1], original.shape[0]))

    # Tampilkan hasil
    plt.figure(figsize=(15, 8))
    plt.subplot(1, 2, 1)
    plt.title('Input Sketch')
    plt.imshow(original)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title('Generated Color')
    plt.imshow(output)
    plt.axis('off')

    # Simpan hasil
    output_path = os.path.join(RESULT_DIR, 'output_' + os.path.basename(image_path))
    plt.savefig(output_path)
    plt.show()

    print("Hasil disimpan di:", output_path)
    display(Image(output_path))

In [ ]:
# Pilih mode: 'train' atau 'test'
MODE = 'train'  # Ganti sesuai kebutuhan

if MODE == 'train':
    print("Memulai training...")
    train_model()
elif MODE == 'test':
    # Upload gambar terlebih dahulu
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        test_image(list(uploaded.keys())[0])
    else:
        print("Silakan upload gambar terlebih dahulu")
else:
    print("Pilih mode 'train' atau 'test'")

Memulai training...
Ditemukan 21 gambar input dan 21 gambar output


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,331,459 (5.08 MB)

 Trainable params: 1,331,459 (5.08 MB)

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 7s/step - loss: 0.0051 - val_loss: 0.0052
Epoch 27/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 0.0050  

2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 7s/step - loss: 0.0050 - val_loss: 0.0052
Epoch 28/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 0.0050  

2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 8s/step - loss: 0.0050 - val_loss: 0.0049
Epoch 29/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 0.0046  

2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 7s/step - loss: 0.0046 - val_loss: 0.0046
Epoch 30/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 0.0044  

2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 7s/step - loss: 0.0043 - val_loss: 0.0044
